In [25]:
import pandas as pd
from datetime import datetime
from pathlib import Path
import spikeinterface.full as si 
from zoneinfo import ZoneInfo


def _read_oephys_datetime(oephys_folder, timezone="Europe/Rome"):
    """Read the datetime of the recording from the sync_messages.txt file in the oephys folder."""

    synch_txt_file = _find_in_folder(oephys_folder, "*/*/*/sync_messages.txt")

    with open(synch_txt_file, "r") as f:
        line = f.readline()

    # get content between : and \n:
    start_msec_utc = int(line.split(": ")[1].split("\n")[0])
    return datetime.fromtimestamp(start_msec_utc / 1000, ZoneInfo(timezone))

def _find_in_folder(folder, pattern):
    return next(folder.glob(pattern))


mid = "M21"
day = "20240421"
ses = "165242"
data_folder = Path(f"/Users/vigji/Desktop/test_mpa_dir/P02_MPAOPTO_LP/e02_ephys-contrapag-stim/{mid}/{day}/{ses}")


# read mouse excel using pandas
mice_df = pd.read_excel("/Users/vigji/Desktop/test_mpa_dir/logs/surgery_log.xlsx",
                        skiprows=1)
mice_df = mice_df.iloc[:, 1:]

subject_id = "D395_M1"
subject_metadata = mice_df.query("subject_id == @subject_id").iloc[0].to_dict()
#Parse birth date:
subject_birth_date = subject_metadata.pop("date_of_birth").replace(tzinfo=ZoneInfo("Europe/Rome"))
oephys_folder = _find_in_folder(data_folder, "NPX/[0-9]*")
session_start_time = _read_oephys_datetime(oephys_folder)

subject_metadata["age"] = f"P{(session_start_time - subject_birth_date).days}D"

/Users/vigji/miniconda3/envs/lab-env/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [26]:
subject_metadata["age"]

'P279D'

In [27]:
subject_birth_date

Timestamp('2023-07-17 00:00:00+0200', tz='Europe/Rome')